In [10]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/stefanocalde/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
inputfile = 'top_2018_01_01.csv'
outputfile = 'turin_2017_07_01.csv'

In [72]:
SIZE = 500000
NAMES = ['device_id', 'recording_date', 'lat', 'lon', 'speed', 'heading', 'gps_quality', 'engine_status',
        'acceleration', 'meters']

i = 0
for chunck in pd.read_csv(inputfile, names=NAMES, chunksize=SIZE, iterator=True, 
                          parse_dates=['recording_date']):
    piece = chunck[chunck.lat.between(45.00678, 45.14021) & chunck.lon.between(7.57785, 7.77337)]
    # piece['loc'] = piece.loc[:, ['lon', 'lat']].apply(lambda c: list(c[0], c[1]), axis=1)
    if i == 0:
        piece.to_csv(outputfile, index=False)
        #piece.to_json('turin_2017_06_01.json', orient='records', date_unit='s')
    else:
        piece.to_csv(outputfile, index=False, mode='a', header=False)
    i += 1
    if i%5 == 0:
        print('Did %d chuncks' %i)

Did 5 chuncks
Did 10 chuncks
Did 15 chuncks
Did 20 chuncks
Did 25 chuncks
Did 30 chuncks
Did 35 chuncks
Did 40 chuncks
Did 45 chuncks
Did 50 chuncks
Did 55 chuncks
Did 60 chuncks
Did 65 chuncks
Did 70 chuncks
Did 75 chuncks
Did 80 chuncks
Did 85 chuncks
Did 90 chuncks
Did 95 chuncks
Did 100 chuncks
Did 105 chuncks
Did 110 chuncks
Did 115 chuncks
Did 120 chuncks
Did 125 chuncks
Did 130 chuncks
Did 135 chuncks
Did 140 chuncks
Did 145 chuncks
Did 150 chuncks
Did 155 chuncks
Did 160 chuncks
Did 165 chuncks
Did 170 chuncks
Did 175 chuncks
Did 180 chuncks
Did 185 chuncks
Did 190 chuncks
Did 195 chuncks
Did 200 chuncks
Did 205 chuncks
Did 210 chuncks
Did 215 chuncks
Did 220 chuncks
Did 225 chuncks
Did 230 chuncks
Did 235 chuncks
Did 240 chuncks


In [6]:
turin_july = pd.read_csv('turin_2017_07_01.csv', parse_dates=['recording_date'])

In [7]:
from datetime import time

turin_july['day'] = turin_july.recording_date.dt.date
turin_july['day_of_the_week'] = turin_july.recording_date.dt.weekday + 1
turin_july['hour'] = turin_july.recording_date.dt.hour
turin_july['hour_half'] = turin_july.recording_date.dt.time.apply(lambda t: time(t.hour, (t.minute // 30)*30))

In [11]:
def special_floor(n):
    return round(floor(n * 400)/400, 4)

def special_floor2(n):
    return round(floor(n * 200)/200, 3)

turin_july['grid_lon'] = turin_july.lon.apply(lambda l: special_floor2(l))
turin_july['grid_lat'] = turin_july.lat.apply(lambda l: special_floor(l))

In [12]:
turin_july['col_number'] = ((turin_july.grid_lon - turin_july.grid_lon.min()) * 200).round().astype(int)
turin_july['row_number'] = ((turin_july.grid_lat - turin_july.grid_lat.min()) * 400).round().astype(int)

tot_cols = turin_july.col_number.max()
turin_july['cell_number'] = (turin_july.row_number*tot_cols) + turin_july.col_number
turin_july['cell_centroid_lat'] = turin_july.grid_lat + 0.00125
turin_july['cell_centroid_lon'] = turin_july.grid_lon + 0.0025

In [27]:
q = 1 - round(50000/ len(cell_density.car_density))/100 - 0.01
c_thres = cell_density.car_density.quantile(q)
len(cell_density[cell_density.car_density >= c_thres])

525

In [24]:
## Valuate cell density
n_cols = tot_cols + 1

cell_density = turin_july[turin_july.engine_status == 1]\
.groupby(['cell_number', 'day'], as_index=False).agg({'device_id': 'nunique'})\
.groupby('cell_number', as_index=False).agg({'device_id': np.mean}).rename(columns={'device_id': 'car_density'})

# Get high density cell numbers and their coordinates

hd_cells = cell_density[cell_density.car_density >= c_thres].cell_number.unique()
print(len(hd_cells))

# Get rid of lonely cells

cell_list = hd_cells.tolist()  # transform to list to remove objects
for c in hd_cells:
    neighs = [c - 1, c + 1, c - n_cols, c + n_cols, c - n_cols + 1, c - n_cols - 1, c + n_cols + 1, c + n_cols - 1]
    s = 0
    for n in neighs:
        s += (hd_cells == n).sum()
    if s == 0:
        cell_list.remove(c)
        
hd_coords = turin_july.loc[turin_july.cell_number.isin(cell_list), ['cell_number','grid_lat', 'grid_lon']]\
.drop_duplicates(subset=['cell_number']).drop('cell_number', 1).values
print(hd_coords.shape)
turin_july.row_number.max(), turin_july.col_number.max()

525
(515, 2)


(54, 39)

In [26]:
## Build the feature vectors
hours = sorted(turin_july.hour_half.unique())

map_h_to_int = dict(zip(hours, range(1, 49)))
map_d_to_int = dict(zip(range(1,8), ['W']*5 + ['ST', 'SN']))
little_turin = turin_july[(turin_july.engine_status == 1) &
                         (turin_july.cell_number.isin(cell_list))].copy()

# Transform weekday and hour
little_turin['day_of_the_week'] = little_turin.day_of_the_week.map(map_d_to_int)
little_turin['hour_half'] = little_turin.hour_half.map(map_h_to_int)

# Compute tl for each weekday, hour and cell

dataset = little_turin\
.groupby(['day_of_the_week', 'hour_half', 'cell_number', 'device_id'], as_index=False).agg({'speed': np.mean})\
.groupby(['day_of_the_week', 'hour_half', 'cell_number'], as_index=False).agg({'device_id': 'nunique', 
                                                                               'speed': np.mean})\
.rename(columns={'device_id': 'n_cars'})
dataset.speed.describe()

count    73917.000000
mean        35.070338
std         23.793653
min          0.000000
25%         20.179795
50%         26.443512
75%         38.945055
max        199.000000
Name: speed, dtype: float64

In [28]:
dataset_10 = dataset.loc[(dataset.hour_half == 10)]

# Select only hours after very early morning (5 AM - 11.30 PM)

dataset = dataset.loc[dataset.hour_half.between(11, 47)]

dataset.loc[dataset.speed != 0, 'TL'] = 1. / dataset.speed
dataset_10.loc[dataset_10.speed != 0, 'TL'] = 1. / dataset_10.speed

In [29]:
# clean row with speed == 0

for c, h in dataset.loc[dataset.speed == 0, ['cell_number', 'hour_half']].values.tolist():
    print(c, h)
    neighs = [c - 1, c + 1, c - 40, c + 40, c - 39, c - 41, c + 41, c + 39]
    d = dataset[dataset.cell_number.isin(neighs)]
    if not len(d):
        print('no neigs for cell %d for hour %d' %(c, h))
    else:
        dataset.loc[(dataset.speed == 0) & (dataset.cell_number == c) & (dataset.hour_half == h),
                   'TL'] = d.TL.mean()
        
for c, h in dataset_10.loc[dataset_10.speed == 0, ['cell_number', 'hour_half']].values.tolist():
    print(c, h)
    neighs = [c - 1, c + 1, c - 40, c + 40, c - 39, c - 41, c + 41, c + 39]
    d = dataset_10[dataset_10.cell_number.isin(neighs)]
    if not len(d):
        print('no neigs for cell %d for hour %d' %(c, h))
    else:
        dataset_10.loc[(dataset_10.speed == 0) & (dataset_10.cell_number == c) & (dataset_10.hour_half == h),
                   'TL'] = d.TL.mean()

482 11
988 11
1538 11
1056 12
521 47
1103 10
1144 10
440 10


In [32]:
def find_neighbours(c):
    return [c - 1, c + 1, c - 40, c + 40, c - 39, c - 41, c + 41, c + 39]

def tl_previous_half(tup):
    d = tup[0]
    h = tup[1]
    c = tup[2]
    if h == 11:
        df = dataset_10
    else:
        df = dataset
    prev_h = h - 1
    if not len(df[(df.cell_number == c) & (df.hour_half == prev_h) & (df.day_of_the_week == d)]):
        n = find_neighbours(c)
        return df[df.cell_number.isin(n) & (df.hour_half == prev_h) & (df.day_of_the_week == d)].TL.mean()
    else:
        return df[(df.cell_number == c) & (df.hour_half == prev_h) & (df.day_of_the_week == d)].TL.values[0]
    
def tl_neighs(tup, main_df, df_10):
    d = tup[0]
    h = tup[1]
    c = tup[2]
    if h == 11:
        df = dataset_10
    else:
        df = main_df
    h = h - 1
    n = find_neighbours(c)
    # print('Did')
    return df[df.cell_number.isin(n) & (df.hour_half == h) & (df.day_of_the_week == d)].TL.mean()

In [ ]:
dataset['tl_neighs'] = dataset.iloc[:, [0, 1, 2]].apply(tl_neighs, args=(dataset, dataset_10), axis=1)

In [33]:
dataset['tl_prev_half'] = dataset.iloc[:, [0, 1, 2]].apply(tl_previous_half, axis=1)

In [35]:
dataset = dataset.merge(turin_july.drop_duplicates(subset=['cell_number'])[['cell_number',
                                                                 'cell_centroid_lat', 'cell_centroid_lon']], 
                        how='left',
             on=['cell_number'])

dataset = dataset.iloc[:, [0, 1, 8, 9, 6, 7, 2, 3, 4, 5]]

In [ ]:
dataset = pd.get_dummies(dataset)
dataset = dataset.iloc[:, [-3, -2, -1] + list(range(9))]
dataset

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

condition = (dataset.tl_neighs < 2) & (dataset.tl_prev_half < 2) & (dataset.n_cars > 6)
X, y = dataset.loc[condition, dataset.columns[:8]].values, np.log(dataset.loc[condition, 
                                                                                       'TL'].values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [65]:
from sklearn.externals import joblib

scaler = joblib.load('scaler_minmax.pkl') 
model = joblib.load('regressor.pkl') 

In [66]:
r2_score(y, model.predict(scaler.transform(X)))

0.9181565591057296

In [69]:
r2_score(np.power(np.e, y), X[:, 7])

0.7308065256231644

In [70]:
dataset.to_csv('july_raw_values.csv', index=False)

In [ ]:
"""ooking for the most discriminative thresholds, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This usually allows to reduce the variance of the model a bit more, at the expense 
of a slightly greater increase in bias"""

"""Like variance, mean squared error has the disadvantage of heavily weighting outliers."""

In [16]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [17]:
db = client.vem

In [18]:
col = db.june2017

In [20]:
!mongoimport -d vem -c june2017 --type csv --file turin_2017_06_01.csv --headerline

2018-07-09T11:06:25.731+0200	connected to: localhost
2018-07-09T11:06:28.714+0200	[........................] vem.june2017	14.3MB/1.07GB (1.3%)
2018-07-09T11:06:31.715+0200	[........................] vem.june2017	29.6MB/1.07GB (2.7%)
2018-07-09T11:06:34.715+0200	[........................] vem.june2017	44.8MB/1.07GB (4.1%)
2018-07-09T11:06:37.714+0200	[#.......................] vem.june2017	59.3MB/1.07GB (5.4%)
2018-07-09T11:06:40.716+0200	[#.......................] vem.june2017	70.9MB/1.07GB (6.5%)
2018-07-09T11:06:43.714+0200	[#.......................] vem.june2017	85.7MB/1.07GB (7.8%)
2018-07-09T11:06:46.714+0200	[##......................] vem.june2017	100MB/1.07GB (9.2%)
2018-07-09T11:06:49.714+0200	[##......................] vem.june2017	115MB/1.07GB (10.5%)
2018-07-09T11:06:52.715+0200	[##......................] vem.june2017	129MB/1.07GB (11.8%)
2018-07-09T11:06:55.714+0200	[###.....................] vem.june2017	145MB/1.07GB (13.2%)
2018-07-09T11:06:58.717+0200	[###...............

In [24]:
j = 0
for doc in col.find({'loc': {'$exists': False}}, projection={'_id', 'lat', 'lon'}):
    col.update_one({'_id': doc['_id']}, {'$push': {'loc': [doc['lon'], doc['lat']]}})
    if j%100000 == 0:
        print('We updated %d docs' %j)
    j += 1

We updated 0 docs
We updated 100000 docs
We updated 200000 docs
We updated 300000 docs
We updated 400000 docs
We updated 500000 docs
We updated 600000 docs
We updated 700000 docs
We updated 800000 docs
We updated 900000 docs
We updated 1000000 docs
We updated 1100000 docs
We updated 1200000 docs
We updated 1300000 docs
We updated 1400000 docs
We updated 1500000 docs
We updated 1600000 docs
We updated 1700000 docs
We updated 1800000 docs
We updated 1900000 docs
We updated 2000000 docs
We updated 2100000 docs
We updated 2200000 docs
We updated 2300000 docs
We updated 2400000 docs
We updated 2500000 docs
We updated 2600000 docs
We updated 2700000 docs
We updated 2800000 docs
We updated 2900000 docs
We updated 3000000 docs
We updated 3100000 docs
We updated 3200000 docs
We updated 3300000 docs
We updated 3400000 docs
We updated 3500000 docs
We updated 3600000 docs
We updated 3700000 docs
We updated 3800000 docs
We updated 3900000 docs
We updated 4000000 docs
We updated 4100000 docs
We upda

In [4]:
db.collection_names()

['june2017']

In [13]:
col.drop()

In [9]:
col.count({'loc': {'$exists': False}})

111668553

In [ ]:
!mongoimport --help

In [12]:
i = 0

for doc in col.find({'loc': {'$exists': False}}):
    col.update_one({'_id': doc['_id']}, {'$push': {'loc': {'LON': doc['lon'], 'LAT': doc['lat']}}})
    i += 1
    if i%10000 == 0:
        print("We did %d documents" %i)

We did 10000 documents
We did 20000 documents
We did 30000 documents
We did 40000 documents
We did 50000 documents
We did 60000 documents
We did 70000 documents
We did 80000 documents
We did 90000 documents
We did 100000 documents
We did 110000 documents
We did 120000 documents
We did 130000 documents
We did 140000 documents
We did 150000 documents
We did 160000 documents
We did 170000 documents
We did 180000 documents
We did 190000 documents
We did 200000 documents
We did 210000 documents
We did 220000 documents
We did 230000 documents
We did 240000 documents
We did 250000 documents
We did 260000 documents
We did 270000 documents
We did 280000 documents
We did 290000 documents
We did 300000 documents
We did 310000 documents
We did 320000 documents
We did 330000 documents
We did 340000 documents
We did 350000 documents
We did 360000 documents
We did 370000 documents
We did 380000 documents
We did 390000 documents
We did 400000 documents
We did 410000 documents
We did 420000 documents
W

We did 3340000 documents
We did 3350000 documents
We did 3360000 documents
We did 3370000 documents
We did 3380000 documents
We did 3390000 documents
We did 3400000 documents
We did 3410000 documents
We did 3420000 documents
We did 3430000 documents
We did 3440000 documents
We did 3450000 documents
We did 3460000 documents
We did 3470000 documents
We did 3480000 documents
We did 3490000 documents
We did 3500000 documents
We did 3510000 documents
We did 3520000 documents
We did 3530000 documents
We did 3540000 documents
We did 3550000 documents
We did 3560000 documents
We did 3570000 documents
We did 3580000 documents
We did 3590000 documents
We did 3600000 documents
We did 3610000 documents
We did 3620000 documents
We did 3630000 documents
We did 3640000 documents
We did 3650000 documents
We did 3660000 documents
We did 3670000 documents
We did 3680000 documents
We did 3690000 documents
We did 3700000 documents
We did 3710000 documents
We did 3720000 documents
We did 3730000 documents


KeyboardInterrupt: 